In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)
pd.set_option("display.max_columns", None)

# Define dictionaries for column names and types
PROCEDURE_COLUMNS = {f"PROCEDIMIENTO{i}": str for i in range(1, 31)}
SERVICE_COLUMNS = {
    "SERVICIOINGRESO": 0,
    "SERVICIOTRASLADO1": 1,
    "SERVICIOTRASLADO2": 2,
    "SERVICIOTRASLADO3": 3,
    "SERVICIOTRASLADO4": 4,
    "SERVICIOTRASLADO5": 5,
    "SERVICIOTRASLADO6": 6,
    "SERVICIOTRASLADO7": 7,
    "SERVICIOTRASLADO8": 8,
    "SERVICIOTRASLADO9": 9,
    "SERVICIOALTA": 10,
}
DATE_COLUMNS = {
    "FECHA_INGRESO": 0,
    "FECHATRASLADO1": 1,
    "FECHATRASLADO2": 2,
    "FECHATRASLADO3": 3,
    "FECHATRASLADO4": 4,
    "FECHATRASLADO5": 5,
    "FECHATRASLADO6": 6,
    "FECHATRASLADO7": 7,
    "FECHATRASLADO8": 8,
    "FECHATRASLADO9": 9,
    "FECHAALTA": 10,
}
BED_TYPES = {
    "AREA MEDICA ADULTO CUIDADOS BASICOS": "Basicos",
    "AREA MEDICA ADULTO CUIDADOS MEDIOS": "Medias",
    "AREA MEDICO-QUIRURGICO CUIDADOS MEDIOS": "Medias",
    "AREA MÉDICA": "Medias",
    "AREA QUIRÚRGICA": "Medias",
    "CIRUGÍA CARDIOVASCULAR": "Medias",
    "UNIDAD DE CUIDADOS INTENSIVOS (UCI) (INDIFERENCIADO)": "UCI",
    "UNIDAD DE CUIDADOS INTENSIVOS ADULTO": "UCI",
    "UNIDAD DE CUIDADOS INTENSIVOS CARDIOLOGÍA": "UCI",
    "UNIDAD DE RECUPERACIÓN DE PABELLONES (CENTRAL Y CMA)": "AMBULATORIO",
    "UNIDAD DE TRATAMIENTO INTERMEDIO (UTI) (INDIFERENCIADO) ADULTO": "UTI",
    "UNIDAD DE TRATAMIENTO INTERMEDIO CARDIOVASCULAR": "UTI",
    "UNIDAD DE TRATAMIENTO INTERMEDIO CIRUGÍA ADULTO": "UTI",
    "UNIDAD DE TRATAMIENTO INTERMEDIO MEDICINA ADULTO": "UTI",
    "UNIDAD DE TRATAMIENTOS INTERMEDIOS MEDICINA": "UTI",
}

# Define relevant diagnoses
RELEVANT_DIAGNOSES = [
    "A15.2",
    "B44.1",
    "C33",
    "C34.0",
    "C34.1",
    "C34.2",
    "C34.3",
    "C38.1",
    "E84.8",
    "I05.1",
    "I05.2",
    "I08.0",
    "I08.1",
    "I23.2",
    "I33.0",
    "I34.0",
    "I35.0",
    "I35.1",
    "I35.2",
    "I36.1",
    "I42.0",
    "I42.1",
    "I49.5",
    "I71.0",
    "I71.1",
    "I71.2",
    "I71.5",
    "J67.9",
    "J84.1",
    "J84.8",
    "J95.5",
    "M05.1",
    "M34.8",
    "Q22.2",
    "Q23.3",
    "Q25.7",
    "Z45.0",
]

In [7]:
df_torax = pd.read_csv("../data/processed/df_procesada_112103.csv", sep=";")
df_torax["FECHAALTA"] = df_torax["FECHAALTA"].str.strip("T00:00:00.000000")

C:\Users\jarojas\AppData\Local\Temp\ipykernel_21196\378652524.py:1: DtypeWarning: Columns (85,86,88,90,129) have mixed types. Specify dtype option on import or set low_memory=False.
  df_torax = pd.read_csv("../data/processed/df_procesada_112103.csv", sep=";")


In [ ]:
# Melt service and date columns
service_long = pd.melt(
    df_torax,
    id_vars=["index", "DIAGNOSTICO1", "ANIO_EGRESO", "IR_29301_SEVERIDAD", "CIP_ENCRIPTADO"],
    value_vars=SERVICE_COLUMNS,
    var_name="service_type",
    value_name="service",
)
date_long = pd.melt(
    df_torax,
    id_vars=["index"],
    value_vars=DATE_COLUMNS,
    var_name="date_type",
    value_name="date",
)

# Merge service and date data
patient_journey = pd.merge(date_long, service_long, how="inner", left_index=True, right_index=True)
patient_journey = patient_journey.drop(columns=["index_y"])
patient_journey = patient_journey.rename(columns={"index_x": "index"})

# Convert service type to categorical
patient_journey["service_type"] = pd.Categorical(
    patient_journey["service_type"], categories=SERVICE_COLUMNS.keys(), ordered=True
)

# Sort and clean data
patient_journey = (
    patient_journey.sort_values(["index", "service_type"]).dropna().reset_index(drop=True)
)
patient_journey["date"] = pd.to_datetime(patient_journey["date"])

# Calculate length of stay
patient_journey["length_of_stay"] = patient_journey.groupby("index")["date"].diff().shift(-1)

# Fix negative and zero length of stay
negative_indices = patient_journey[patient_journey["length_of_stay"] < pd.Timedelta(0)].index + 1
patient_journey.loc[negative_indices, "date"] += pd.offsets.DateOffset(years=1)
patient_journey["length_of_stay"] = patient_journey.groupby("index")["date"].diff().shift(-1)

zero_indices = patient_journey[patient_journey["length_of_stay"] == pd.Timedelta(0)].index
patient_journey.loc[zero_indices, "length_of_stay"] += pd.Timedelta(days=1)

# Map bed types
patient_journey["service"] = patient_journey["service"].replace(BED_TYPES)

# Summarize length of stay by diagnosis, bed type, and year
summary = (
    patient_journey.groupby(["ANIO_EGRESO", "DIAGNOSTICO1", "IR_29301_SEVERIDAD", "service"]).agg(
        dias_estada_totales=("length_of_stay", "sum"),
        n_pacientes=("CIP_ENCRIPTADO", "nunique"),
        n_egresos=("index", "nunique"),
    )
).reset_index()

# Calculates the percentage of occupancy per bed type
summary["porcentaje_ocupacion"] = summary["dias_estada_totales"] / summary.groupby(
    ["ANIO_EGRESO", "DIAGNOSTICO1"]
)["dias_estada_totales"].transform("sum")

# Calculates length of stay per pacient
summary["dias_estada_por_paciente"] = summary["dias_estada_totales"] / summary["n_pacientes"]

# Filter for relevant diagnoses
summary_relevant = summary.query("DIAGNOSTICO1.isin(@RELEVANT_DIAGNOSES)")

ValueError: unconverted data remains when parsing with format "%Y-%m-%d": "T00:00:00.000000", at position 1. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [ ]:
# Save to Excel
with pd.ExcelWriter("../data/interim/resumen_ocupacion_por_diagnostico.xlsx") as writer:
    summary_relevant.to_excel(writer, sheet_name="mas_relevantes", index=False)
    summary.to_excel(writer, sheet_name="todos", index=False)

In [ ]:
# Obtiene el resumen de dias de estada totales por diagnostico
resumen_dias_de_estada = (
    patient_journey.groupby(["ANIO_EGRESO", "DIAGNOSTICO1", "service"])["length_of_stay"]
    .sum()
    .unstack(fill_value=pd.Timedelta(0))
)

# Total dias de estada por anio y diag
total_dias_de_estada = resumen_dias_de_estada.sum(axis=1)

# Agrega el total de dias ocupados al DataFrame original
resumen_dias_de_estada["total_dias"] = total_dias_de_estada

# Obtiene el porcentaje de ocupacion por cama
porcentaje_de_ocupacion = resumen_dias_de_estada.div(total_dias_de_estada, axis=0)
porcentaje_de_ocupacion = (
    porcentaje_de_ocupacion.stack()
    .to_frame("porcentaje_de_ocupacion")
    .query("porcentaje_de_ocupacion > 0")
)

# Convierte los dias totales a formato long
resumen_dias_de_estada = resumen_dias_de_estada.stack().to_frame("dias_estada_totales")

# Filtra todos los tipos de camas que carezcan de dias
resumen_dias_de_estada = resumen_dias_de_estada[
    resumen_dias_de_estada["dias_estada_totales"] > pd.Timedelta(0)
]

# Une el porcentaje de ocupacion
resumen_dias_de_estada = resumen_dias_de_estada.merge(
    porcentaje_de_ocupacion, how="inner", left_index=True, right_index=True
).reset_index()

# Obtiene el resumen en formato wide
resumen_dias_de_estada_wide = (
    resumen_dias_de_estada.set_index(["ANIO_EGRESO", "DIAGNOSTICO1", "service"])
    .unstack()
    .reset_index(level=1)
)

In [ ]:
resumen_dias_de_estada_wide_relevantes = resumen_dias_de_estada_wide[
    resumen_dias_de_estada_wide[("DIAGNOSTICO1", "")].isin(RELEVANT_DIAGNOSES)
]

In [ ]:
with pd.ExcelWriter("../data/interim/resumen_ocupacion_por_diagnostico_DE.xlsx") as file:
    resumen_dias_de_estada.to_excel(file, sheet_name="long")
    resumen_dias_de_estada_wide.to_excel(file, sheet_name="wide")
    resumen_dias_de_estada_wide_relevantes.to_excel(file, sheet_name="wide_relevantes")